In [1]:
import os
import warnings
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage.io import imread,imshow
from skimage.transform import resize
from skimage.color import rgb2grey
from sklearn.utils import shuffle
from skimage import feature
import cv2

In [14]:
path = [r"dataset/train/with_mask",r"dataset/train/without_mask"]
imSize = (100,100)
mask_img = []
nomask_img = []

for folder in os.listdir(path[0]):
    directory = os.path.join(path[0], folder)
    
    for img_path in os.listdir(directory):
        pth = os.path.join(directory, img_path)
        img = cv2.imread(pth,0)
        img = cv2.resize(img, imSize)
        img = cv2.equalizeHist(img)
        mask_img.append(img)

for folder in os.listdir(path[1]):
    directory = os.path.join(path[1], folder)
    
    for img_path in os.listdir(directory):
        pth = os.path.join(directory, img_path)
        img = cv2.imread(pth,0)
        img = cv2.resize(img, imSize)
        nomask_img.append(img)

In [19]:
fig, ax = plt.subplots(50,4,sharex=True,sharey=True,figsize = (10,10))
for i, axi in enumerate(ax.flat):
    axi.imshow(mask_img[i])
    axi.set_title('Label : {}'.format(yfit[i]),)

In [3]:
nMask = len(mask_img)
nNoMask = len(nomask_img)

if nMask < nNoMask:
    nImages = nMask
else:
    nImages = nNoMask

print(nImages)

1851


In [4]:
nCol = mask_img[0].shape[0] * mask_img[0].shape[1]

#dimensions of images are (x * y) right now,
#need to convert it into array of xy*1
#basically flattening the 40 by 40 matrix into a 1600 by 1 matrix
#putting all the rows together in a single row

for i in range(nImages):
    mask_img[i]     = np.ndarray.flatten(mask_img[i]).reshape(nCol,1)
    nomask_img[i]   = np.ndarray.flatten(nomask_img[i])
    
    mask_img[i]     = np.transpose(mask_img[i])
    nomask_img[i]   = np.transpose(nomask_img[i])
    
mask_img   = np.dstack(mask_img)
nomask_img = np.dstack(mask_img)

mask_img   = np.rollaxis(mask_img, axis = 2, start = 0)
nomask_img = np.rollaxis(nomask_img, axis = 2, start = 0)

mask_img   = mask_img.reshape(nImages, nCol)
nomask_img = nomask_img.reshape(nImages, nCol)

maskData   = pd.DataFrame(mask_img)
nomaskData = pd.DataFrame(nomask_img)

maskData["label"]   = "MASK"
nomaskData["label"] = "NOMASK"

totalData = pd.concat([maskData, nomaskData])

output = shuffle(totalData).reset_index()
output = output.drop(['index'], axis=1)


In [5]:
x = output.values[:,:-1]
y = output.values[:,-1]

In [6]:
from sklearn.decomposition import PCA                      # principle component analysis, decompose face into its features
from sklearn.svm import SVC                                # support vector classifier, the ml models class
from sklearn.pipeline import make_pipeline                 # support function for GridSearchCV
from sklearn.model_selection import train_test_split       # split training and testing data
from sklearn.model_selection import GridSearchCV           # func to find the best parameters
from sklearn import metrics                                # accuracy finder

In [7]:
#principle component analysis
#decomposes the face into its individual features.
# n_components = number of components the face will be split into
# whiten = ??
# random_state = 0
pca = PCA(n_components=40, whiten=False, random_state=0)

#support vector classifier.
#draws a straigt line b/w two classes, i.e. classifies the classes into two parts for classification
svc = SVC(kernel='rbf', class_weight='balanced')

model = make_pipeline(pca, svc)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=0)#splitting training and testing data.

In [9]:
params = {'svc__C': [x for x in range(1,5)],'svc__gamma': [0.001, 0.005, 0.006,0.01,0.05,0.06,0.004,0.04]}

grid = GridSearchCV(model, params)
%time grid.fit(x_train, y_train)
print(grid.best_params_)

Wall time: 11min 43s
{'svc__C': 1, 'svc__gamma': 0.001}


In [10]:
model = grid.best_estimator_
yfit = model.predict(x_test)

In [11]:
print(x_test.shape)

(926, 10000)


In [12]:
fig, ax = plt.subplots(4,4,sharex=True,sharey=True,figsize = (10,10))
for i, axi in enumerate(ax.flat):
    axi.imshow(x_test[i].reshape((imSize)).astype(np.float64),cmap = "gray",interpolation = "nearest")
    axi.set_title('Label : {}'.format(yfit[i]),)

In [13]:
print(metrics.accuracy_score(y_test,yfit))

0.5010799136069114
